In [ ]:
https://github.com/pytorch/ao/tree/main/torchao/optim
https://pytorch.org/blog/pytorch-native-architecture-optimization/
https://arxiv.org/pdf/2502.10940

# Project Index

[Custom Model Notebook](../../../notebooks/custom_model.ipynb)  
[Training Notebook](../../../notebooks/train.ipynb)  
[Project Config Notebook](../../../notebooks/project_config.ipynb)  
[Forgather Notebook](../../../notebooks/forgather.ipynb)  

In [ ]:
import forgather.nb.notebooks as nb

nb.display_project_index(config_template="adamw.yaml", show_pp_config=True, show_generated_code=False)

In [ ]:
from forgather.ml.trainer import Trainer
from forgather.ml.trainer_types import TrainingArguments
from forgather.ml.training_script import TrainingScript
from pprint import pp
import torch
from forgather.project import Project
import forgather.nb.notebooks as nb

# Load default baseline config
proj = Project("adamw.yaml")